## Install Glove

In [2]:
!pip install git+https://github.com/millawell/bias-ml-dh.git#subdirectory=material/notebooks/bias_ml_dh_utils
!pip install --upgrade tqdm
!git clone https://github.com/stanfordnlp/GloVe.git
%cd GloVe
!make
%cd ..

  Cloning https://github.com/millawell/bias-ml-dh.git to /private/var/folders/jp/c1yk1wvn6x58ftj04zg1z8680000gn/T/pip-req-build-5le5duur
  Running command git clone -q https://github.com/millawell/bias-ml-dh.git /private/var/folders/jp/c1yk1wvn6x58ftj04zg1z8680000gn/T/pip-req-build-5le5duur
  Created wheel for bias-ml-dh-utils: filename=bias_ml_dh_utils-0.1-py3-none-any.whl size=2211 sha256=40f6b477a4a98047430dc86aaf261a00b2c5bd7da73f4dec8c12c5054a9605e9
  Stored in directory: /private/var/folders/jp/c1yk1wvn6x58ftj04zg1z8680000gn/T/pip-ephem-wheel-cache-n3k8tx3l/wheels/d4/8d/af/a27ef01a2dc2a7313cfcbb128b906b77e986b55a6ebd3d52a7
Successfully built bias-ml-dh-utils
Requirement already up-to-date: tqdm in /Users/davidlassner/Envs/wh4/lib/python3.7/site-packages (4.43.0)
fatal: destination path 'GloVe' already exists and is not an empty directory.
/Users/davidlassner/research/conferences_and_papers/workshops/2020 DHd/website/material/notebooks/GloVe
mkdir -p build
/Users/davidlassner/rese

## Download the aggressive wikipedia comments data set

In [3]:
!mkdir data
!wget https://ndownloader.figshare.com/files/7394506 -O agression_annotation.tsv
!wget https://ndownloader.figshare.com/files/7038038 -O aggression_annotated_comments.tsv
!mv agression_annotation.tsv data/
!mv aggression_annotated_comments.tsv data/

mkdir: data: File exists
--2020-02-29 15:25:34--  https://ndownloader.figshare.com/files/7394506
Resolving ndownloader.figshare.com (ndownloader.figshare.com)...18.203.184.13, 54.246.162.108, 34.252.157.212, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|18.203.184.13|:443...connected.
HTTP request sent, awaiting response...302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/7394506/aggression_annotations.tsv [following]
--2020-02-29 15:25:34--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/7394506/aggression_annotations.tsv
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.56.3
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.56.3|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 30627328 (29M) [binary/octet-stream]
Saving to: ‘agression_annotation.tsv’

agression_annotatio 100%[===================>]  29.21M  41.1MB/s    in 0.7s    

2020-02-29 15:25:35 (41.1 

## Preprocess data

In [12]:
import pandas as pd
from tqdm import tqdm
aggression_annotated_comments = pd.read_csv("data/aggression_annotated_comments.tsv", sep="\t")
agression_annotation = pd.read_csv("data/agression_annotation.tsv", sep="\t")

agression_data = pd.merge(aggression_annotated_comments, agression_annotation, on="rev_id")

In [16]:
comments = []
labels = []

def regex_filter(comment):
    return comment.replace("NEWLINE_TOKEN", "\t")

for rev_id, rev in tqdm(agression_data.groupby("rev_id")):
    comments.append(regex_filter(rev.iloc[0].comment))
    labels.append(rev.aggression.sum()/len(rev) >.5)

100%|██████████| 115864/115864 [00:59<00:00, 1951.27it/s]


In [17]:
from sklearn.model_selection import train_test_split    
train_embeddings, X_other, embeddings_labels, y_other  = train_test_split(comments, labels, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X_other, y_other, random_state=456)

In [18]:
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [19]:
with open("data/all_comments_plain.txt", "w") as all_fout:
    with open("data/aggressive_comments_plain.txt", "w") as agg_fout:
        with open("data/non_aggressive_comments_plain.txt", "w") as non_agg_fout:
            for comment, label in tqdm(
                zip(tokenizer.pipe(train_embeddings), embeddings_labels),
                total=len(train_embeddings),
                desc="storing comments"
            ):
                out_line = " ".join([t.text.replace("\n", "\t") for t in comment])
                all_fout.write("{}\n".format(out_line))
                if label == 1:
                    agg_fout.write("{}\n".format(out_line))
                else:
                    non_agg_fout.write("{}\n".format(out_line))

storing comments: 100%|██████████| 86898/86898 [01:03<00:00, 1358.15it/s]


In [20]:
!ls data

agg_vec_data.tsv
agg_vec_meta.tsv
aggression_annotated_comments.tsv
aggressive_comments_cooccurrences.bin
aggressive_comments_cooccurrences_shuffled.bin
aggressive_comments_plain.txt
aggressive_comments_vec.txt
agression_annotation.tsv
all_comments_cooccurrences.bin
all_comments_cooccurrences_shuffled.bin
all_comments_plain.txt
all_comments_vec.txt
all_vec_data.tsv
all_vec_meta.tsv
noagg_vec_data.tsv
noagg_vec_meta.tsv
non_aggressive_comments_cooccurrences.bin
non_aggressive_comments_cooccurrences_shuffled.bin
non_aggressive_comments_plain.txt
non_aggressive_comments_vec.txt
twitter
vocab.txt


In [21]:
!ls GloVe/build/

common.o      cooccur.o     glove.o       shuffle.o     vocab_count.o
cooccur       glove         shuffle       vocab_count


## Build Vocabulary

In [9]:
!gloVe/build/vocab_count -max-vocab 25000 -min-count 10 < data/all_comments_plain.txt > data/vocab.txt 

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 tokens.6100000 tokens.6200000 tokens.6300000 tokens.6400000 tokens.6500000 to

## Create cooccurrences

In [10]:
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/non_aggressive_comments_plain.txt > data/non_aggressive_comments_cooccurrences.bin
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/aggressive_comments_plain.txt > data/aggressive_comments_cooccurrences.bin
!gloVe/build/cooccur -vocab-file data/vocab.txt < data/all_comments_plain.txt > data/all_comments_cooccurrences.bin

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 10485784
overflow length: 28521267
Reading vocab from file "data/vocab.txt"...loaded 22657 words.
Building lookup table...table contains 51262847 elements.
Processing token: 0100000200000300000400000500000600000700000800000900000100000011000001200000130000014000001500000160000017000001800000190000020000002100000220000023000002400000250000026000002700000280000029000003000000310000032000003300000340000035000003600000370000038000003900000400000041000004200000430000044000004500000460000047000004800000490000050000005100000520000053000005400000550000056000005700000580000059000006000000610000062000006300000640000065000006600000Processed 6668532 tokens.
Writing cooccurrences to disk........2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1

In [11]:
!gloVe/build/shuffle -verbose 2 -memory 4 < data/non_aggressive_comments_cooccurrences.bin > data/non_aggressive_comments_cooccurrences_shuffled.bin
!gloVe/build/shuffle -verbose 2 -memory 4 < data/aggressive_comments_cooccurrences.bin > data/aggressive_comments_cooccurrences_shuffled.bin
!gloVe/build/shuffle -verbose 2 -memory 4 < data/all_comments_cooccurrences.bin > data/all_comments_cooccurrences_shuffled.bin

Using random seed 1580483054
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 15671104 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.15671104 lines.Merging temp files: processed 15671104 lines.

Using random seed 1580483061
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 2843613 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.2843613 lines.Merging temp files: processed 2843613 lines.

Using random seed 1580483062
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 16595758 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.16595758 lines.Merging temp files: processed 16595758 lines.



## Train Glove

In [12]:
!gloVe/build/glove -input-file data/non_aggressive_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/non_aggressive_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2
!gloVe/build/glove -input-file data/aggressive_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/aggressive_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2
!gloVe/build/glove -input-file data/all_comments_cooccurrences_shuffled.bin -vocab-file data/vocab.txt -save-file data/all_comments_vec -verbose 2 -vector-size 32 -threads 8 -alpha 0.75 -x-max 100.0 -eta 0.05 -binary 0 -model 2

TRAINING MODEL
Read 15671104 lines.
Initializing parameters...Using random seed 1580483070
done.
vector size: 32
vocab size: 22657
x_max: 100.000000
alpha: 0.750000
01/31/20 - 04:04.33PM, iter: 001, cost: 0.022223
01/31/20 - 04:04.34PM, iter: 002, cost: 0.014752
01/31/20 - 04:04.36PM, iter: 003, cost: 0.013140
01/31/20 - 04:04.37PM, iter: 004, cost: 0.011990
01/31/20 - 04:04.39PM, iter: 005, cost: 0.011016
01/31/20 - 04:04.41PM, iter: 006, cost: 0.010252
01/31/20 - 04:04.42PM, iter: 007, cost: 0.009696
01/31/20 - 04:04.44PM, iter: 008, cost: 0.009289
01/31/20 - 04:04.45PM, iter: 009, cost: 0.008974
01/31/20 - 04:04.47PM, iter: 010, cost: 0.008727
01/31/20 - 04:04.49PM, iter: 011, cost: 0.008527
01/31/20 - 04:04.50PM, iter: 012, cost: 0.008361
01/31/20 - 04:04.52PM, iter: 013, cost: 0.008222
01/31/20 - 04:04.53PM, iter: 014, cost: 0.008103
01/31/20 - 04:04.55PM, iter: 015, cost: 0.008001
01/31/20 - 04:04.57PM, iter: 016, cost: 0.007908
01/31/20 - 04:04.59PM, iter: 017, cost: 0.007828
01

## Load Embeddings

In [22]:
from bisect import bisect_left
import numpy as np
import os

with open("data/vocab.txt") as fin:
    vocab,_ = zip(*map(lambda x: x.split(" "), fin))
vocab = sorted(list(vocab) + ["<unk>"])

def save_for_tf_projector(embedding, vocab, outdir, identifier):
    out_path_data = os.path.join(outdir, "{}_data.tsv".format(identifier))
    out_path_meta = os.path.join(outdir, "{}_meta.tsv".format(identifier))

    with open(out_path_data, "w") as fout:
        for row in embedding:
            fout.write("{}\n".format("\t".join(map(str, row.tolist()))))
    
    with open(out_path_meta, "w") as fout:
        for word in vocab:
            fout.write("{}\n".format(word.encode("utf-8")))

In [23]:
def index(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

def load_embedding(path_, vocab):
    dim = 32
    mat = np.empty((len(vocab), dim))
    
    with open(path_) as fin:
        for row in tqdm(fin):
            splitted = row.replace("\n", "").split(" ")
            key, vec = splitted[0], splitted[1:]
            mat[index(vocab, key)] = vec
            
    return mat
            
all_vec = load_embedding("data/all_comments_vec.txt", vocab)
aggressive_vec = load_embedding("data/aggressive_comments_vec.txt", vocab)
non_aggressive_vec = load_embedding("data/non_aggressive_comments_vec.txt", vocab)

save_for_tf_projector(all_vec, vocab, "data", "all_vec")
save_for_tf_projector(aggressive_vec, vocab, "data", "agg_vec")
save_for_tf_projector(non_aggressive_vec, vocab, "data", "noagg_vec")

22658it [00:00, 94949.18it/s]
22658it [00:00, 97662.94it/s]
22658it [00:00, 92751.19it/s]
